In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [42]:
df = pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [43]:
df.info()
#no null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [63]:
#Lets create test and train set
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.05,random_state=42)
train_set=[]
test_set=[]
for train_index , test_index in split.split(df,df['HeartDisease']):
    train_set = df.iloc[train_index]
    test_set = df.iloc[test_index]
#making traing set
train_labels = train_set['HeartDisease']
train_set = train_set.drop('HeartDisease',axis=1)

#lets convert test set to csv to check output
test_set.to_csv('heart_model_test_set.csv',index=False)

In [64]:
train_set.columns.tolist()

['Age',
 'Sex',
 'ChestPainType',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'RestingECG',
 'MaxHR',
 'ExerciseAngina',
 'Oldpeak',
 'ST_Slope']

In [65]:
#Now lets create attribute for traing
num_attribute = ['Age',
    'RestingBP',
 'Cholesterol',
 'FastingBS',
 'MaxHR',
 'Oldpeak']
cat_attribute = ['Sex',
 'ChestPainType',
'RestingECG',
 'ExerciseAngina',
 'ST_Slope']

In [66]:
#Now lets create nummarical and catigorycal pipe line
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([('standardscaler',StandardScaler())])

from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([('onehot',OneHotEncoder(handle_unknown='ignore'))])

In [67]:
#Now lets make full pipeline
from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
                            ('num',num_pipeline,num_attribute),
                            ('cat',cat_pipeline,cat_attribute)    
])
full_pipeline #it looks cool

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [69]:
#Lets see the new df
pd.DataFrame(heart_prepared,
    columns=full_pipeline.get_feature_names_out(),
    index=train_set.index)

,num__Age,num__RestingBP,num__Cholesterol,num__FastingBS,num__MaxHR,num__Oldpeak,cat__Sex_F,cat__Sex_M,cat__ChestPainType_ASY,cat__ChestPainType_ATA,cat__ChestPainType_NAP,cat__ChestPainType_TA,cat__RestingECG_LVH,cat__RestingECG_Normal,cat__RestingECG_ST,cat__ExerciseAngina_N,cat__ExerciseAngina_Y,cat__ST_Slope_Down,cat__ST_Slope_Flat,cat__ST_Slope_Up
212,0.254748,-0.132426,0.696442,-0.543775,-0.334853,0.121370,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
90,-0.488906,-1.207621,0.072839,-0.543775,0.909485,-0.832097,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
368,0.360984,0.405171,-1.834653,-0.543775,-0.645937,1.074838,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
891,1.317110,0.727729,0.714784,-0.543775,0.598400,-0.832097,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
541,2.379472,-1.530179,-0.798371,-0.543775,-0.645937,2.505039,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-0.807615,-0.670024,0.274593,-0.543775,0.520629,-0.832097,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
180,-0.170197,-0.132426,0.228740,-0.543775,-0.645937,1.074838,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
577,1.423346,0.727729,1.549312,-0.543775,-1.034792,0.979491,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
252,0.785929,-0.401225,0.843173,-0.543775,-0.840365,-0.832097,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [85]:
#Now lets train the model
from sklearn.ensemble import RandomForestClassifier
forest_cla = RandomForestClassifier()

full_pipeline_model = Pipeline([
    ('preprocessor', full_pipeline),       # ColumnTransformer
    ('classifier', forest_cla)             # RandomForestClassifier
])
full_pipeline_model.fit(train_set, train_labels)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [86]:
#lets get the model out for easier access
import joblib
joblib.dump(full_pipeline_model, "heart_disease_model.pkl")

['heart_disease_model.pkl']

In [88]:
#NOW WE WE WILL TEST IT YOU CAN USE THIS CELL OR OTHER NOTEBOOK NAMED prediction.ipynb JUST NEED TO FEED OWN DATA OR ROW NUMBER FROM heart_model_test_set.csv


model = joblib.load("heart_disease_model.pkl")
RowToBePredicted = 22 #change row number here which you want to be predicted from heart_model_test_set.csv
new_data_prepared = test_set.iloc[RowToBePredicted-1:RowToBePredicted]  

# Get prediction
prediction = model.predict(new_data_prepared)
if prediction == 0:
    print('0:NO HEART DISEASE')
else: print('1:HAVE HEART DISEASE')

0:NO HEART DISEASE
